In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from kerastuner.tuners import RandomSearch
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
# Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime
# Clear any logs from previous runs
!rm -rf ./logs/ 

from kerastuner.tuners import RandomSearch

train_df = pd.read_csv('data/features_small_provinces.csv')

train_df = train_df[train_df.metroscubiertos <= train_df.metrostotales]

train_x = train_df.drop(columns=['precio', 'usosmultiples'])
train_y = train_df.precio

scaler = MinMaxScaler(feature_range = (0,1))
coordinates_scaler = MinMaxScaler(feature_range = (-1, 1))

train_x.antiguedad = scaler.fit_transform(np.array(train_x.antiguedad).reshape(-1, 1))
train_x.metroscubiertos = scaler.fit_transform(np.array(train_x.metroscubiertos).reshape(-1, 1))
train_x.metrostotales = scaler.fit_transform(np.array(train_x.metrostotales).reshape(-1, 1))
train_x.habitaciones = scaler.fit_transform(np.array(train_x.habitaciones).reshape(-1, 1))
train_x.garages = scaler.fit_transform(np.array(train_x.garages).reshape(-1, 1))
train_x.banos = scaler.fit_transform(np.array(train_x.banos).reshape(-1, 1))

n_cols = train_x.shape[0]

train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
#train_x.head()
train_x.shape

(696, 15)

In [2]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq=1)

In [3]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units_1', min_value=64, max_value=100, step=16), activation = 'relu', input_shape=(15,)))
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=64,
                                            max_value=100,
                                            step=16),
                               activation='relu'))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mae'])
    return model

In [4]:
tuner = RandomSearch(
    build_model,
    objective='mae',
    max_trials=5,
    executions_per_trial=1,
    directory='models/deep_model_small_provinces',
    project_name='deep_learning_with_keras_tuner')

tuner.search_space_summary()

tuner.search(x=train_x,
             y=train_y,
             epochs=20,
             validation_split=0.5,
            callbacks = [tensorboard_callback])

tuner.results_summary()

Train on 348 samples, validate on 348 samples
Epoch 1/20
348/348 [==============================] - ETA: 5s - loss: 3511062364160.0000 - mae: 1294906.250 - 1s 2ms/sample - loss: 2750228642898.3911 - mae: 1239739.8750 - val_loss: 3060524803083.7700 - val_mae: 1316727.0000
Epoch 2/20
348/348 [==============================] - ETA: 0s - loss: 1747195396096.0000 - mae: 1094405.250 - 0s 203us/sample - loss: 2750223809806.7129 - mae: 1239738.0000 - val_loss: 3060513877403.9541 - val_mae: 1316723.2500
Epoch 3/20
348/348 [==============================] - ETA: 0s - loss: 1349067472896.0000 - mae: 936870.62 - 0s 189us/sample - loss: 2750202720773.8848 - mae: 1239729.8750 - val_loss: 3060466456458.2993 - val_mae: 1316706.6250
Epoch 4/20
348/348 [==============================] - ETA: 0s - loss: 4166227066880.0000 - mae: 1546447.500 - 0s 197us/sample - loss: 2750120154453.3335 - mae: 1239696.8750 - val_loss: 3060275446889.9312 - val_mae: 1316640.5000
Epoch 5/20
348/348 [==========================

Train on 348 samples, validate on 348 samples
Epoch 1/20
348/348 [==============================] - ETA: 3s - loss: 2785682653184.0000 - mae: 1265729.750 - 0s 1ms/sample - loss: 2750229105416.8276 - mae: 1239740.0000 - val_loss: 3060526056553.9312 - val_mae: 1316727.5000
Epoch 2/20
348/348 [==============================] - ETA: 0s - loss: 1457184178176.0000 - mae: 990780.75 - 0s 225us/sample - loss: 2750225812044.5059 - mae: 1239738.7500 - val_loss: 3060518719535.0806 - val_mae: 1316725.0000
Epoch 3/20
348/348 [==============================] - ETA: 0s - loss: 2200380112896.0000 - mae: 1078184.500 - 0s 213us/sample - loss: 2750208492461.6089 - mae: 1239732.2500 - val_loss: 3060471548680.8276 - val_mae: 1316708.6250
Epoch 4/20
348/348 [==============================] - ETA: 0s - loss: 2532711071744.0000 - mae: 1274575.500 - 0s 190us/sample - loss: 2750092076420.4141 - mae: 1239691.2500 - val_loss: 3060191958545.6553 - val_mae: 1316613.2500
Epoch 5/20
348/348 [==========================

Train on 348 samples, validate on 348 samples
Epoch 1/20
348/348 [==============================] - ETA: 3s - loss: 3103273779200.0000 - mae: 1310906.250 - 0s 1ms/sample - loss: 2750229209370.4829 - mae: 1239740.0000 - val_loss: 3060526903248.9194 - val_mae: 1316727.7500
Epoch 2/20
348/348 [==============================] - ETA: 0s - loss: 1816526848000.0000 - mae: 1149093.500 - 0s 190us/sample - loss: 2750228883950.3447 - mae: 1239739.8750 - val_loss: 3060526806828.1377 - val_mae: 1316727.6250
Epoch 3/20
348/348 [==============================] - ETA: 0s - loss: 3028806795264.0000 - mae: 1243843.500 - 0s 156us/sample - loss: 2750228698641.6553 - mae: 1239739.8750 - val_loss: 3060526568789.3335 - val_mae: 1316727.6250
Epoch 4/20
348/348 [==============================] - ETA: 0s - loss: 3079782531072.0000 - mae: 1302624.750 - 0s 191us/sample - loss: 2750228489227.7700 - mae: 1239739.7500 - val_loss: 3060526306645.3335 - val_mae: 1316727.5000
Epoch 5/20
348/348 [========================

Train on 348 samples, validate on 348 samples
Epoch 1/20
348/348 [==============================] - ETA: 2s - loss: 3079885553664.0000 - mae: 1295951.125 - 0s 1ms/sample - loss: 2750228457589.7012 - mae: 1239739.7500 - val_loss: 3060524935662.3447 - val_mae: 1316727.1250
Epoch 2/20
348/348 [==============================] - ETA: 0s - loss: 4290623045632.0000 - mae: 1417686.500 - 0s 153us/sample - loss: 2750225513742.7129 - mae: 1239738.6250 - val_loss: 3060519924794.8506 - val_mae: 1316725.3750
Epoch 3/20
348/348 [==============================] - ETA: 0s - loss: 3213655277568.0000 - mae: 1461784.000 - 0s 151us/sample - loss: 2750218431334.9883 - mae: 1239735.8750 - val_loss: 3060507230396.3218 - val_mae: 1316721.0000
Epoch 4/20
348/348 [==============================] - ETA: 0s - loss: 2346401136640.0000 - mae: 1193961.750 - 0s 175us/sample - loss: 2750200130971.9541 - mae: 1239729.2500 - val_loss: 3060475532064.3677 - val_mae: 1316710.1250
Epoch 5/20
348/348 [========================

Train on 348 samples, validate on 348 samples
Epoch 1/20
348/348 [==============================] - ETA: 2s - loss: 3617795342336.0000 - mae: 1510312.500 - 0s 1ms/sample - loss: 2750074823126.8047 - mae: 1239682.2500 - val_loss: 3059425741765.1494 - val_mae: 1316354.6250
Epoch 2/20
348/348 [==============================] - ETA: 0s - loss: 1332635369472.0000 - mae: 971863.75 - 0s 153us/sample - loss: 2742592445428.2300 - mae: 1236805.6250 - val_loss: 3026674931782.6206 - val_mae: 1305280.8750
Epoch 3/20
348/348 [==============================] - ETA: 0s - loss: 2572941262848.0000 - mae: 1163648.000 - 0s 158us/sample - loss: 2632653001692.6895 - mae: 1194948.5000 - val_loss: 2708442437325.9771 - val_mae: 1192601.1250
Epoch 4/20
348/348 [==============================] - ETA: 0s - loss: 1740095094784.0000 - mae: 1117771.250 - 0s 203us/sample - loss: 1969196570506.2988 - mae: 920879.5625 - val_loss: 1392145341804.8735 - val_mae: 681839.7500
Epoch 5/20
348/348 [============================

INFO:tensorflow:Oracle triggered exit


In [5]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16994), started 0:02:46 ago. (Use '!kill 16994' to kill it.)

In [9]:
models = tuner.get_best_models(num_models=2)

In [10]:
best_model_1 = models[0]

In [11]:
best_model_2 = models[1]

In [12]:
best_model_1.fit(train_x, train_y, epochs = 500, validation_split = 0.5, callbacks = [tensorboard_callback])

Train on 348 samples, validate on 348 samples
Epoch 1/500
348/348 [==============================] - ETA: 2s - loss: 602702086144.0000 - mae: 545211.500 - 0s 1ms/sample - loss: 654142857875.1265 - mae: 470493.2500 - val_loss: 544743107430.9886 - val_mae: 454627.6875
Epoch 2/500
348/348 [==============================] - ETA: 0s - loss: 643926523904.0000 - mae: 482232.343 - 0s 159us/sample - loss: 653365209205.7012 - mae: 453365.4688 - val_loss: 541728347289.0114 - val_mae: 441220.5938
Epoch 3/500
348/348 [==============================] - ETA: 0s - loss: 228768481280.0000 - mae: 340328.750 - 0s 161us/sample - loss: 643301802360.6437 - mae: 458692.1875 - val_loss: 539559504554.6667 - val_mae: 450061.8750
Epoch 4/500
348/348 [==============================] - ETA: 0s - loss: 548564828160.0000 - mae: 454175.968 - 0s 126us/sample - loss: 639703818710.8046 - mae: 462374.5625 - val_loss: 536173393896.4598 - val_mae: 430849.4375
Epoch 5/500
348/348 [==============================] - ETA: 0s -

Epoch 74/500
348/348 [==============================] - ETA: 0s - loss: 245303803904.0000 - mae: 345672.187 - 0s 164us/sample - loss: 585021981048.6437 - mae: 411735.5938 - val_loss: 512620833262.3448 - val_mae: 411248.1875
Epoch 75/500
348/348 [==============================] - ETA: 0s - loss: 593045684224.0000 - mae: 496622.375 - 0s 159us/sample - loss: 602042938238.5288 - mae: 453659.5938 - val_loss: 513360691011.6782 - val_mae: 402783.8750
Epoch 76/500
348/348 [==============================] - ETA: 0s - loss: 746088693760.0000 - mae: 511718.531 - 0s 138us/sample - loss: 596871691346.3907 - mae: 411322.1250 - val_loss: 511528982786.9425 - val_mae: 399338.0312
Epoch 77/500
348/348 [==============================] - ETA: 0s - loss: 2638007500800.0000 - mae: 704331.43 - 0s 168us/sample - loss: 613842033275.5862 - mae: 444246.7500 - val_loss: 514980932266.6667 - val_mae: 400511.8125
Epoch 78/500
348/348 [==============================] - ETA: 0s - loss: 717434716160.0000 - mae: 458737.

Epoch 111/500
348/348 [==============================] - ETA: 0s - loss: 241022582784.0000 - mae: 365630.875 - 0s 155us/sample - loss: 554609105719.9081 - mae: 403709.0000 - val_loss: 532444790784.0000 - val_mae: 412126.4688
Epoch 112/500
348/348 [==============================] - ETA: 0s - loss: 406274244608.0000 - mae: 338792.562 - 0s 151us/sample - loss: 595588444489.5632 - mae: 445850.8438 - val_loss: 533407539105.8391 - val_mae: 410525.5312
Epoch 113/500
348/348 [==============================] - ETA: 0s - loss: 531658276864.0000 - mae: 465143.375 - 0s 154us/sample - loss: 572321892752.1840 - mae: 400359.8125 - val_loss: 530321344912.1839 - val_mae: 408145.5938
Epoch 114/500
348/348 [==============================] - ETA: 0s - loss: 139600363520.0000 - mae: 260416.687 - 0s 162us/sample - loss: 543371890358.4368 - mae: 409121.8438 - val_loss: 535106067067.5862 - val_mae: 424634.7500
Epoch 115/500
348/348 [==============================] - ETA: 0s - loss: 995863691264.0000 - mae: 60

348/348 [==============================] - ETA: 0s - loss: 406304063488.0000 - mae: 370984.687 - 0s 157us/sample - loss: 516198669111.9081 - mae: 392814.7500 - val_loss: 571278790138.1150 - val_mae: 428777.4375
Epoch 184/500
348/348 [==============================] - ETA: 0s - loss: 251631517696.0000 - mae: 322136.312 - 0s 167us/sample - loss: 514467408554.6666 - mae: 408109.1875 - val_loss: 576377517679.8160 - val_mae: 426433.5625
Epoch 185/500
348/348 [==============================] - ETA: 0s - loss: 400992370688.0000 - mae: 387266.875 - 0s 159us/sample - loss: 517929827598.7126 - mae: 386650.8438 - val_loss: 574089065154.2069 - val_mae: 425623.0000
Epoch 186/500
348/348 [==============================] - ETA: 0s - loss: 343011033088.0000 - mae: 411599.343 - 0s 157us/sample - loss: 507442134851.6782 - mae: 398143.6250 - val_loss: 587891107769.3793 - val_mae: 444765.2812
Epoch 187/500
348/348 [==============================] - ETA: 0s - loss: 264014659584.0000 - mae: 330683.281 - 0s 

348/348 [==============================] - ETA: 0s - loss: 304593371136.0000 - mae: 329797.062 - 0s 138us/sample - loss: 462839405273.7471 - mae: 385104.8125 - val_loss: 626299315211.7701 - val_mae: 435756.4688
Epoch 256/500
348/348 [==============================] - ETA: 0s - loss: 371557335040.0000 - mae: 399481.375 - 0s 165us/sample - loss: 457686281439.6321 - mae: 372054.3750 - val_loss: 621275215848.4597 - val_mae: 438565.6562
Epoch 257/500
348/348 [==============================] - ETA: 0s - loss: 361683681280.0000 - mae: 379599.937 - 0s 167us/sample - loss: 457747174929.6552 - mae: 372259.9688 - val_loss: 637746058393.0115 - val_mae: 451574.8125
Epoch 258/500
348/348 [==============================] - ETA: 0s - loss: 589804601344.0000 - mae: 489815.375 - 0s 149us/sample - loss: 461808426913.8391 - mae: 396851.4062 - val_loss: 636606531230.8966 - val_mae: 440668.5938
Epoch 259/500
348/348 [==============================] - ETA: 0s - loss: 206998290432.0000 - mae: 275565.312 - 0s 

Epoch 328/500
348/348 [==============================] - ETA: 0s - loss: 567953326080.0000 - mae: 455759.375 - 0s 147us/sample - loss: 394880134932.5977 - mae: 374996.9688 - val_loss: 719245709359.0804 - val_mae: 457565.2812
Epoch 329/500
348/348 [==============================] - ETA: 0s - loss: 173579042816.0000 - mae: 279606.625 - 0s 145us/sample - loss: 399432801833.1954 - mae: 358421.0625 - val_loss: 732823782882.5748 - val_mae: 483618.5312
Epoch 330/500
348/348 [==============================] - ETA: 0s - loss: 431487385600.0000 - mae: 425750.375 - 0s 169us/sample - loss: 399829586155.4023 - mae: 364321.5000 - val_loss: 731392761385.1954 - val_mae: 469575.6250
Epoch 331/500
348/348 [==============================] - ETA: 0s - loss: 744793243648.0000 - mae: 577543.125 - 0s 150us/sample - loss: 383686149696.7356 - mae: 370618.7188 - val_loss: 719986462767.0804 - val_mae: 462335.1875
Epoch 332/500
348/348 [==============================] - ETA: 0s - loss: 73820143616.0000 - mae: 206

348/348 [==============================] - ETA: 0s - loss: 106103119872.0000 - mae: 231508.828 - 0s 158us/sample - loss: 186961302116.0460 - mae: 293470.4062 - val_loss: 1088377362690.9425 - val_mae: 549234.5625
Epoch 401/500
348/348 [==============================] - ETA: 0s - loss: 238264614912.0000 - mae: 304171.875 - 0s 162us/sample - loss: 192226125000.0919 - mae: 290016.3125 - val_loss: 1064152206277.1494 - val_mae: 554868.4375
Epoch 402/500
348/348 [==============================] - ETA: 0s - loss: 149830107136.0000 - mae: 259888.484 - 0s 145us/sample - loss: 181940899133.7931 - mae: 281197.7500 - val_loss: 1101449624470.0688 - val_mae: 554752.0000
Epoch 403/500
348/348 [==============================] - ETA: 0s - loss: 79185895424.0000 - mae: 210359.95 - 0s 158us/sample - loss: 195548770727.7242 - mae: 287908.2188 - val_loss: 1094555713347.6781 - val_mae: 551205.3125
Epoch 404/500
348/348 [==============================] - ETA: 0s - loss: 229792481280.0000 - mae: 265105.843 - 0

348/348 [==============================] - ETA: 0s - loss: 54470688768.0000 - mae: 181119.62 - 0s 145us/sample - loss: 99347021824.0000 - mae: 211997.1719 - val_loss: 1272740968989.4253 - val_mae: 588190.5000
Epoch 473/500
348/348 [==============================] - ETA: 0s - loss: 181346729984.0000 - mae: 276044.250 - 0s 149us/sample - loss: 95377095950.7126 - mae: 211830.6250 - val_loss: 1295477897757.4253 - val_mae: 585752.4375
Epoch 474/500
348/348 [==============================] - ETA: 0s - loss: 82503761920.0000 - mae: 184631.93 - 0s 147us/sample - loss: 91332135123.8621 - mae: 203436.8438 - val_loss: 1291686896275.1265 - val_mae: 586774.7500
Epoch 475/500
348/348 [==============================] - ETA: 0s - loss: 117528846336.0000 - mae: 215857.218 - 0s 125us/sample - loss: 94837564286.5287 - mae: 209528.1094 - val_loss: 1274057765005.2415 - val_mae: 582954.9375
Epoch 476/500
348/348 [==============================] - ETA: 0s - loss: 50308218880.0000 - mae: 159037.12 - 0s 156us/

In [13]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16994), started 0:05:41 ago. (Use '!kill 16994' to kill it.)

In [3]:
# Best Model hyperparams
#|-learning_rate: 0.01
#|-num_layers: 3
#|-units_0: 80
#|-units_1: 80
#|-units_2: 64
#|-units_3: 80
#|-units_4: 64
!rm -rf ./logs
tuned_model = keras.Sequential()
tuned_model.add(layers.Dense(80, activation = 'relu', input_shape = (15,)))
tuned_model.add(layers.Dropout(0.5))
tuned_model.add(layers.Dense(80, activation = 'relu'))
tuned_model.add(layers.Dropout(0.4))
tuned_model.add(layers.Dense(64, activation = 'relu'))
tuned_model.add(layers.Dropout(0.3))
tuned_model.add(layers.Dense(80, activation = 'relu'))
tuned_model.add(layers.Dropout(0.2))
tuned_model.add(layers.Dense(64, activation = 'relu'))
tuned_model.add(layers.Dropout(0.2))
tuned_model.add(layers.Dense(1))
tuned_model.compile(learning_rate = 0.01, loss = 'mean_squared_error', metrics = ['mae'])
tuned_model.fit(x=train_x, y=train_y, epochs=300, validation_split=0.5, callbacks = [tensorboard_callback])
# Clear any logs from previous runs
%tensorboard --logdir logs/fit

Train on 348 samples, validate on 348 samples
Epoch 1/300
348/348 [==============================] - ETA: 10s - loss: 3259642413056.0000 - mae: 1325656.25 - 1s 4ms/sample - loss: 2750226719002.4829 - mae: 1239739.1250 - val_loss: 3060518960587.0347 - val_mae: 1316725.0000
Epoch 2/300
348/348 [==============================] - ETA: 0s - loss: 2680418992128.0000 - mae: 1097371.250 - 0s 196us/sample - loss: 2750197235335.3564 - mae: 1239728.3750 - val_loss: 3060437271092.9653 - val_mae: 1316696.8750
Epoch 3/300
348/348 [==============================] - ETA: 0s - loss: 2158651113472.0000 - mae: 1174433.500 - 0s 186us/sample - loss: 2750007809177.0117 - mae: 1239653.7500 - val_loss: 3059986494899.4946 - val_mae: 1316542.7500
Epoch 4/300
348/348 [==============================] - ETA: 0s - loss: 2055072645120.0000 - mae: 1185436.250 - 0s 182us/sample - loss: 2749157909704.0918 - mae: 1239333.8750 - val_loss: 3058151559215.0806 - val_mae: 1315920.6250
Epoch 5/300
348/348 [===================

Epoch 37/300
348/348 [==============================] - ETA: 0s - loss: 2698821238784.0000 - mae: 757398.81 - 0s 152us/sample - loss: 914417402797.6093 - mae: 599110.3750 - val_loss: 815398562851.3103 - val_mae: 563242.3750
Epoch 38/300
348/348 [==============================] - ETA: 0s - loss: 409587089408.0000 - mae: 527023.312 - 0s 169us/sample - loss: 900293372586.6667 - mae: 601417.6250 - val_loss: 814663408628.2299 - val_mae: 554859.3125
Epoch 39/300
348/348 [==============================] - ETA: 0s - loss: 633855410176.0000 - mae: 466662.875 - 0s 169us/sample - loss: 847517073972.9656 - mae: 567278.0000 - val_loss: 789838558266.8506 - val_mae: 561671.9375
Epoch 40/300
348/348 [==============================] - ETA: 0s - loss: 366995243008.0000 - mae: 476726.875 - 0s 176us/sample - loss: 876932815295.2643 - mae: 589918.5000 - val_loss: 783386263740.3219 - val_mae: 557097.1250
Epoch 41/300
348/348 [==============================] - ETA: 0s - loss: 3061412003840.0000 - mae: 929320

Epoch 74/300
348/348 [==============================] - ETA: 0s - loss: 420598054912.0000 - mae: 433849.062 - 0s 181us/sample - loss: 726389120553.1954 - mae: 511331.4062 - val_loss: 625158786554.1150 - val_mae: 486479.3438
Epoch 75/300
348/348 [==============================] - ETA: 0s - loss: 879319973888.0000 - mae: 641625.500 - 0s 186us/sample - loss: 798958956308.5978 - mae: 534488.5000 - val_loss: 633788483607.5403 - val_mae: 475496.9062
Epoch 76/300
348/348 [==============================] - ETA: 0s - loss: 616024571904.0000 - mae: 578994.500 - 0s 185us/sample - loss: 795065810567.3563 - mae: 545788.3125 - val_loss: 635186992092.6897 - val_mae: 472577.1562
Epoch 77/300
348/348 [==============================] - ETA: 0s - loss: 559621210112.0000 - mae: 424256.250 - 0s 174us/sample - loss: 747939679738.1150 - mae: 498732.7812 - val_loss: 620417144714.2988 - val_mae: 480903.1875
Epoch 78/300
348/348 [==============================] - ETA: 0s - loss: 287217221632.0000 - mae: 456916.

Epoch 111/300
348/348 [==============================] - ETA: 0s - loss: 806146670592.0000 - mae: 510025.750 - 0s 173us/sample - loss: 722805433544.0919 - mae: 517851.0312 - val_loss: 570093981743.0804 - val_mae: 442690.1562
Epoch 112/300
348/348 [==============================] - ETA: 0s - loss: 733367631872.0000 - mae: 522832.312 - 0s 183us/sample - loss: 711457466109.0575 - mae: 489965.2500 - val_loss: 570539920701.7931 - val_mae: 441617.3438
Epoch 113/300
348/348 [==============================] - ETA: 0s - loss: 283312586752.0000 - mae: 322398.125 - 0s 160us/sample - loss: 731033161963.4022 - mae: 488769.7500 - val_loss: 562695178263.5403 - val_mae: 444931.1250
Epoch 114/300
348/348 [==============================] - ETA: 0s - loss: 833295351808.0000 - mae: 512657.625 - 0s 165us/sample - loss: 728131518652.3218 - mae: 494952.1875 - val_loss: 563385444093.0575 - val_mae: 443456.0000
Epoch 115/300
348/348 [==============================] - ETA: 0s - loss: 1005279379456.0000 - mae: 6

348/348 [==============================] - ETA: 0s - loss: 274044157952.0000 - mae: 376322.343 - 0s 175us/sample - loss: 707389159965.4253 - mae: 471777.2500 - val_loss: 541174945674.2988 - val_mae: 424801.3750
Epoch 148/300
348/348 [==============================] - ETA: 0s - loss: 230046826496.0000 - mae: 353862.500 - 0s 180us/sample - loss: 700915704090.4828 - mae: 486268.6562 - val_loss: 544089442139.2184 - val_mae: 423515.1875
Epoch 149/300
348/348 [==============================] - ETA: 0s - loss: 598290202624.0000 - mae: 503796.906 - 0s 171us/sample - loss: 678931457035.7701 - mae: 485508.7500 - val_loss: 539179652437.3333 - val_mae: 425812.3750
Epoch 150/300
348/348 [==============================] - ETA: 0s - loss: 2188290555904.0000 - mae: 668124.75 - 0s 169us/sample - loss: 703785996711.7241 - mae: 470366.1562 - val_loss: 539597732381.4253 - val_mae: 424673.6875
Epoch 151/300
348/348 [==============================] - ETA: 0s - loss: 2547160186880.0000 - mae: 759349.50 - 0s 

348/348 [==============================] - ETA: 0s - loss: 344719228928.0000 - mae: 446068.031 - 0s 196us/sample - loss: 706415662609.6552 - mae: 488346.5625 - val_loss: 525793811820.8735 - val_mae: 418552.9688
Epoch 184/300
348/348 [==============================] - ETA: 0s - loss: 281912082432.0000 - mae: 406659.562 - 0s 175us/sample - loss: 729889260649.9310 - mae: 495363.5938 - val_loss: 531459159663.8161 - val_mae: 416727.7188
Epoch 185/300
348/348 [==============================] - ETA: 0s - loss: 2263939547136.0000 - mae: 643691.75 - 0s 179us/sample - loss: 689101351218.0229 - mae: 459939.0312 - val_loss: 526101527069.4253 - val_mae: 417671.3438
Epoch 186/300
348/348 [==============================] - ETA: 0s - loss: 562968395776.0000 - mae: 429464.625 - 0s 178us/sample - loss: 710599199873.4712 - mae: 483004.0938 - val_loss: 525208746337.1035 - val_mae: 418577.0000
Epoch 187/300
348/348 [==============================] - ETA: 0s - loss: 435364364288.0000 - mae: 418127.437 - 0s 

348/348 [==============================] - ETA: 0s - loss: 546671362048.0000 - mae: 416112.531 - 0s 186us/sample - loss: 726479789232.5516 - mae: 479447.0312 - val_loss: 521137211038.8965 - val_mae: 414929.5625
Epoch 220/300
348/348 [==============================] - ETA: 0s - loss: 2464339722240.0000 - mae: 707457.81 - 0s 175us/sample - loss: 695395358225.6552 - mae: 485734.5312 - val_loss: 523527574869.3333 - val_mae: 414098.8125
Epoch 221/300
348/348 [==============================] - ETA: 0s - loss: 682193453056.0000 - mae: 561613.500 - 0s 163us/sample - loss: 732107613384.0919 - mae: 493142.9062 - val_loss: 527183560139.0345 - val_mae: 413669.0938
Epoch 222/300
348/348 [==============================] - ETA: 0s - loss: 468557365248.0000 - mae: 349715.343 - 0s 155us/sample - loss: 679378640048.5516 - mae: 451516.1875 - val_loss: 518696978267.2184 - val_mae: 415599.5312
Epoch 223/300
348/348 [==============================] - ETA: 0s - loss: 942441431040.0000 - mae: 585128.312 - 0s 

348/348 [==============================] - ETA: 0s - loss: 99460677632.0000 - mae: 235865.53 - 0s 164us/sample - loss: 682929027990.0690 - mae: 476401.9375 - val_loss: 521214761407.2643 - val_mae: 411367.8750
Epoch 256/300
348/348 [==============================] - ETA: 0s - loss: 709108432896.0000 - mae: 522221.250 - 0s 200us/sample - loss: 688981330696.8276 - mae: 473606.2188 - val_loss: 511404641221.1494 - val_mae: 412521.1875
Epoch 257/300
348/348 [==============================] - ETA: 0s - loss: 211686686720.0000 - mae: 333992.531 - 0s 176us/sample - loss: 671915445565.7931 - mae: 460204.6562 - val_loss: 510854554259.1265 - val_mae: 412629.4375
Epoch 258/300
348/348 [==============================] - ETA: 0s - loss: 641503592448.0000 - mae: 536519.500 - 0s 159us/sample - loss: 705531404193.8391 - mae: 464825.2500 - val_loss: 512667959107.6782 - val_mae: 412315.7812
Epoch 259/300
348/348 [==============================] - ETA: 0s - loss: 520418918400.0000 - mae: 394834.437 - 0s 16

348/348 [==============================] - ETA: 0s - loss: 449379958784.0000 - mae: 371803.218 - 0s 182us/sample - loss: 694361695055.4484 - mae: 458248.8125 - val_loss: 512100033547.7701 - val_mae: 409779.0000
Epoch 292/300
348/348 [==============================] - ETA: 0s - loss: 712112930816.0000 - mae: 565655.625 - 0s 175us/sample - loss: 704118260088.6437 - mae: 469432.6875 - val_loss: 520249275639.1724 - val_mae: 409230.2500
Epoch 293/300
348/348 [==============================] - ETA: 0s - loss: 383441240064.0000 - mae: 384675.031 - 0s 188us/sample - loss: 715545660922.1150 - mae: 476444.3750 - val_loss: 527128780705.8391 - val_mae: 409659.9062
Epoch 294/300
348/348 [==============================] - ETA: 0s - loss: 100205895680.0000 - mae: 236200.703 - 0s 177us/sample - loss: 683351299789.9771 - mae: 450746.4062 - val_loss: 511122871260.6896 - val_mae: 410033.2812
Epoch 295/300
348/348 [==============================] - ETA: 0s - loss: 425516892160.0000 - mae: 382408.343 - 0s 

Reusing TensorBoard on port 6006 (pid 13146), started 3 days, 1:00:53 ago. (Use '!kill 13146' to kill it.)

In [4]:
test_df = pd.read_csv('data/test_small_provinces_modified.csv')
test_id = test_df.id
test_df = test_df.drop(columns=['id', 'usosmultiples'])
#test_df.antiguedad = scaler.fit_transform(np.array(test_df.antiguedad).reshape(-1, 1))
#test_df.metroscubiertos = scaler.fit_transform(np.array(test_df.metroscubiertos).reshape(-1, 1))
#test_df.metrostotales = scaler.fit_transform(np.array(test_df.metrostotales).reshape(-1, 1))
#test_df.habitaciones = scaler.fit_transform(np.array(test_df.habitaciones).reshape(-1, 1))
#test_df.garages = scaler.fit_transform(np.array(test_df.garages).reshape(-1, 1))
#test_df.banos = scaler.fit_transform(np.array(test_df.banos).reshape(-1, 1))
test_df = test_df.to_numpy()

In [5]:
tuned_model_prediction = tuned_model.predict(test_df)

ValueError: Error when checking input: expected dense_input to have shape (15,) but got array with shape (16,)

In [6]:
prediction = tuned_model.predict(train_x)

In [6]:
prediction

array([[1471112.2 ],
       [1525055.6 ],
       [ 563851.8 ],
       [1242546.4 ],
       [1325111.  ],
       [1921076.9 ],
       [ 446258.1 ],
       [ 902801.56],
       [ 740866.4 ],
       [1416326.9 ],
       [ 644496.25],
       [2480247.5 ],
       [2288037.5 ],
       [ 315806.9 ],
       [1755442.  ],
       [1155330.8 ],
       [1079785.8 ],
       [ 379200.8 ],
       [2224810.5 ],
       [ 430990.4 ],
       [ 355249.9 ],
       [ 349252.5 ],
       [2689153.8 ],
       [ 601380.  ],
       [ 404526.66],
       [ 418033.7 ],
       [2520592.  ],
       [ 328255.34],
       [ 708386.1 ],
       [ 553520.56],
       [1134750.4 ],
       [ 356281.44],
       [ 589321.2 ],
       [1886140.8 ],
       [ 743205.3 ],
       [1421487.6 ],
       [ 633412.94],
       [2790108.2 ],
       [ 686965.8 ],
       [ 518003.8 ],
       [1912045.6 ],
       [ 299813.28],
       [ 428579.78],
       [ 409579.9 ],
       [1989099.8 ],
       [ 642730.56],
       [1279524.4 ],
       [25912

In [7]:
a = train_y - prediction
a

array([[ -321112.25   ,   208887.75   ,  -991112.25   , ...,
        -1011112.25   , -1081112.25   ,  -711112.25   ],
       [ -375055.625  ,   154944.375  , -1045055.625  , ...,
        -1065055.625  , -1135055.625  ,  -765055.625  ],
       [  586148.1875 ,  1116148.1875 ,   -83851.8125 , ...,
         -103851.8125 ,  -173851.8125 ,   196148.1875 ],
       ...,
       [  800678.8125 ,  1330678.8125 ,   130678.8125 , ...,
          110678.8125 ,    40678.8125 ,   410678.8125 ],
       [  754032.90625,  1284032.90625,    84032.90625, ...,
           64032.90625,    -5967.09375,   364032.90625],
       [ -921940.375  ,  -391940.375  , -1591940.375  , ...,
        -1611940.375  , -1681940.375  , -1311940.375  ]])

In [7]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(prediction, train_y)

413828.2098823635

In [8]:
# serialize model to JSON
model_json = tuned_model.to_json()
with open("tuned_model_small_provinces.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
tuned_model.save_weights("model_small_provinces.h5")
print("Saved model to disk")

Saved model to disk
